In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from os import system
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
import copy
from MyDataLoader import MyDataLoader
from torch.utils.data import DataLoader
from model import *
from utils import *
from train import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SOS_token = 0
EOS_token = 1
# ----------Hyper Parameters----------#
input_size = 28  # The number of vocabulary: input_size==vocab_size  {SOS,EOS,a,b,c,....,z}
hidden_size = 256  # LSTM hidden size
latent_size = 32
conditional_size = 8
max_length=15
file_path= 'best_CVAE_18_ckpt'  #'score0.84.pt'

if __name__ == '__main__':
    # dataloader
    dataset_test = MyDataLoader(path='lab5_dataset/test.txt', train=False)
    loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False)

    # VAE model
    CVAE = CVAE(input_size, hidden_size, latent_size, conditional_size, max_length).to(device)
    CVAE.load_state_dict(torch.load(os.path.join('models',file_path)))

    """
    evaluate
    """
    total_BLEUscore=0
    total_Gaussianscore=0
    test_time=1
    for i in range(test_time):
        conversion, BLEUscore = evaluate(CVAE, loader_test, dataset_test.tensor2string)
        print('test.txt prediction:')
        for i in range(len(conversion)):
            print('input:',conversion[i][0])
            print('target:',conversion[i][1])
            print('prediction:',conversion[i][2])
            print()
        total_BLEUscore+=BLEUscore

    print(f'avg BLEUscore {total_BLEUscore/test_time:.2f}')
    
    for i in range(test_time):    
        # generate words
        generated_words=Gaussian_generate(CVAE,latent_size,dataset_test.tensor2string)
        Gaussianscore=Gaussian_score(generated_words)
        print('generate 100 words with 4 different tenses:')
        print(generated_words)
        total_Gaussianscore+=Gaussianscore
    print()
    print(f'avg Gaussianscore {total_Gaussianscore/test_time:.2f}')

test.txt prediction:
input: abandon
target: abandoned
prediction: abandoned

input: abet
target: abetting
prediction: abetting

input: begin
target: begins
prediction: begins

input: expend
target: expends
prediction: expends

input: sent
target: sends
prediction: sents

input: split
target: splitting
prediction: splitting

input: flared
target: flare
prediction: flare

input: functioning
target: function
prediction: function

input: functioning
target: functioned
prediction: functionalized

input: healing
target: heals
prediction: heals

avg BLEUscore 0.88
generate 100 words with 4 different tenses:
[['bock', 'combines', 'combining', 'combined'], ['yamper', 'yampers', 'yeaping', 'yamped'], ['exhaler', 'exhalerates', 'exhaltrating', 'exhaluted'], ['approve', 'thansudes', 'apprecializing', 'thanked'], ['bohe', 'bohs', 'bohed', 'bows'], ['forewarn', 'forewarns', 'overslepting', 'overflowed'], ['enstroll', 'entreats', 'entreaching', 'entergoied'], ['disguast', 'disquats', 'distaphing', 'd